# 🤖 Stage 5: PyCaret需要予測モデル構築

## 目的
**機械学習で売上を予測し、精度の高い発注を実現する**

このノートブックでは、PyCaretを使って需要予測モデルを構築します。

### PyCaretとは？
**ローコードで機械学習モデルを構築できるPythonライブラリ**
- 複数のアルゴリズムを自動で比較
- 最適なモデルを自動で選択
- ハイパーパラメータの自動調整
- 少ないコードで高度な予測が可能

### 分析内容
1. **データ準備**
   - 特徴量エンジニアリング
   - 訓練データ・テストデータの分割

2. **モデル構築（回帰問題）**
   - PyCaretで複数モデルを自動比較
   - 最適モデルの選択
   - モデルのチューニング

3. **モデル評価**
   - 予測精度の確認
   - 特徴量の重要度分析
   - エラー分析

4. **予測の実行**
   - 未来の売上を予測
   - 発注推奨量の算出

### 業務的な意義
- **発注担当**: 勘ではなく、AIによる科学的な予測で発注できる
- **店長**: 欠品・廃棄を最小化し、利益を最大化できる
- **オーナー**: データドリブンな経営判断が可能になる

---

## 📝 注意事項
このノートブックは基本的なPyCaretの使い方を示すものです。
実際の需要予測システムは「動的売上予測システム - PyCaret 3.ipynb」で構築します。

---

## 📦 1. 環境準備

In [ ]:
# 日本語フォント設定（共通モジュール）
import warnings
warnings.filterwarnings('ignore')

# よく使うライブラリを先に読み込む
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ウィジェットの有無を通知・フラグ化
try:
    import ipywidgets as widgets
    from IPython.display import display, HTML, clear_output
    WIDGETS_AVAILABLE = True
    print('✅ ipywidgets利用可能')
except Exception:
    WIDGETS_AVAILABLE = False
    print('⚠️ ipywidgets未インストール - 一部機能制限')

import font_setup
JP_FP = font_setup.setup_fonts(show_test=False)


In [2]:
import pandas as pd
from datetime import datetime, timedelta

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# PyCaretのインポート
from pycaret.regression import *
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print("✅ PyCaret回帰モジュールをインポートしました")

✅ PyCaret回帰モジュールをインポートしました


## 📁 2. データ読み込み

In [3]:
file_path = 'output/06_final_enriched_20250701_20250930.csv'

print(f"📂 データ読み込み中: {file_path}")
df = pd.read_csv(file_path, encoding='utf-8-sig', low_memory=False)
df['日付'] = pd.to_datetime(df['日付'])

print(f"\n✅ 読み込み完了")
print(f"   データ期間: {df['日付'].min().strftime('%Y-%m-%d')} 〜 {df['日付'].max().strftime('%Y-%m-%d')}")
print(f"   データ件数: {len(df):,}行")
print(f"   特徴量数: {len(df.columns)}列")

📂 データ読み込み中: output/06_final_enriched_20250701_20250930.csv

✅ 読み込み完了
   データ期間: 2025-07-01 〜 2025-09-30
   データ件数: 83,789行
   特徴量数: 132列


## 🎯 3. 予測対象の選定

### 業務的な方針
全商品を一度に予測するのは計算コストが高いため、重要な商品（Aランク）に絞って予測する

In [4]:
# 商品別売上集計
product_sales = df.groupby('商品名')['売上数量'].sum().sort_values(ascending=False)

# 上位10商品を選択（デモ用）
top_products = product_sales.head(10).index.tolist()

print("="*60)
print("🎯 予測対象商品（売上上位10商品）")
print("="*60)
for i, product in enumerate(top_products, 1):
    sales = product_sales[product]
    print(f"  {i}. {product}: {sales:,.0f}個")

print("\n💡 実務では、Aランク商品全てを対象にします")

🎯 予測対象商品（売上上位10商品）
  1. レジ袋: 12,785個
  2. ◎ジャスミン茶６００: 10,729個
  3. い・ろ・は・す５４０: 10,093個
  4. ◎緑茶６００: 9,270個
  5. ◎天然水新潟県津南６００: 7,227個
  6. 直巻　焼しゃけ: 6,468個
  7. 直巻　和風ツナマヨネーズ: 6,360個
  8. 手巻シーチキンマヨネーズ: 5,338個
  9. レジ袋１２号バイオマス: 5,249個
  10. サントリー天然水　５５０: 4,984個

💡 実務では、Aランク商品全てを対象にします


## 🔧 4. データ準備（1商品のデモ）

### 業務的な方針
まず1商品でモデルを構築し、うまくいったら全商品に展開する

In [5]:
# デモ商品を選択（売上1位の商品）
demo_product = top_products[0]

print(f"📊 デモ商品: {demo_product}")

# その商品のデータを抽出
df_product = df[df['商品名'] == demo_product].copy()

# 日付でソート
df_product = df_product.sort_values('日付').reset_index(drop=True)

print(f"\nデータ件数: {len(df_product):,}行")
print(f"期間: {df_product['日付'].min().strftime('%Y-%m-%d')} 〜 {df_product['日付'].max().strftime('%Y-%m-%d')}")

📊 デモ商品: レジ袋

データ件数: 276行
期間: 2025-07-01 〜 2025-09-30


## 📊 5. 特徴量の選択

### 業務的な方針
予測に有用な特徴量のみを選択し、ノイズを減らす

In [6]:
# 予測に使わない列（ID系・テキスト系）
exclude_cols = ['日付', '商品名', '店舗', 'フェイスくくり大分類', 'フェイスくくり中分類', 'フェイスくくり小分類']

# 目的変数
target = '売上数量'

# 特徴量（数値型のみ）
feature_cols = [col for col in df_product.select_dtypes(include=[np.number]).columns 
                if col not in exclude_cols and col != target]

print("="*60)
print("📊 使用する特徴量")
print("="*60)
print(f"\n特徴量数: {len(feature_cols)}個")
print(f"\n主な特徴量:")
for i, col in enumerate(feature_cols[:20], 1):
    print(f"  {i}. {col}")

if len(feature_cols) > 20:
    print(f"  ... 他 {len(feature_cols) - 20}個")

print(f"\n目的変数: {target}")

📊 使用する特徴量

特徴量数: 124個

主な特徴量:
  1. 売上金額
  2. 年
  3. 月
  4. 日
  5. 曜日
  6. 週番号
  7. 年内日数
  8. 祝日フラグ
  9. 土曜フラグ
  10. 日曜フラグ
  11. 週末フラグ
  12. 平日フラグ
  13. 休日フラグ
  14. 休日タイプ
  15. 休日前日
  16. 休日翌日
  17. 休日前々日
  18. 連休日数
  19. 連休中日番号
  20. 連休フラグ
  ... 他 104個

目的変数: 売上数量


## 🔀 6. 訓練データ・テストデータの分割

### 時系列データの分割方法
時系列データでは、**過去のデータで訓練し、未来のデータでテストする**

- 訓練データ: 最初の80%
- テストデータ: 最後の20%

In [7]:
# 訓練データとテストデータの分割点
split_point = int(len(df_product) * 0.8)

train_data = df_product.iloc[:split_point].copy()
test_data = df_product.iloc[split_point:].copy()

print("="*60)
print("🔀 データ分割")
print("="*60)
print(f"\n訓練データ: {len(train_data):,}行")
print(f"  期間: {train_data['日付'].min().strftime('%Y-%m-%d')} 〜 {train_data['日付'].max().strftime('%Y-%m-%d')}")

print(f"\nテストデータ: {len(test_data):,}行")
print(f"  期間: {test_data['日付'].min().strftime('%Y-%m-%d')} 〜 {test_data['日付'].max().strftime('%Y-%m-%d')}")

print("\n💡 訓練データで学習 → テストデータで精度を検証")

🔀 データ分割

訓練データ: 220行
  期間: 2025-07-01 〜 2025-09-12

テストデータ: 56行
  期間: 2025-09-12 〜 2025-09-30

💡 訓練データで学習 → テストデータで精度を検証


## 🤖 7. PyCaretセットアップ

### 業務的な意義
PyCaretが自動でデータの前処理・特徴量エンジニアリングを行う

In [8]:
print("="*60)
print("🤖 PyCaretセットアップ中...")
print("="*60)
print("\n※ この処理には数分かかる場合があります")

# PyCaretのセットアップ
try:
    # 特徴量と目的変数を含むデータフレームを作成
    train_features = train_data[feature_cols + [target]].copy()
    
    # 欠損値を除去
    train_features = train_features.dropna()
    
    print(f"\n訓練データ（前処理後）: {len(train_features):,}行")
    
    # PyCaretセットアップ
    reg = setup(
        data=train_features,
        target=target,
        session_id=123,  # 再現性のため
        verbose=False,   # 詳細ログを抑制
             # 警告を抑制
        html=False       # HTML出力を無効化
    )
    
    print("\n✅ PyCaretセットアップ完了")
    
except Exception as e:
    print(f"\n❌ エラー: {e}")
    print("\n💡 PyCaretがインストールされていない可能性があります")
    print("   インストール: pip install pycaret")

🤖 PyCaretセットアップ中...

※ この処理には数分かかる場合があります

訓練データ（前処理後）: 200行

✅ PyCaretセットアップ完了


## 🏆 8. モデルの比較・選択

### 業務的な意義
PyCaretが複数のアルゴリズムを自動で試し、最も精度の高いモデルを見つける

In [ ]:
print("="*60)
print("🏆 モデル比較中...")
print("="*60)
print("\n※ 複数のアルゴリズムを試すため、数分かかります")

try:
    # 複数モデルを比較（上位5つのみ表示）
    best_models = compare_models(n_select=5, verbose=False)
    
    print("\n✅ モデル比較完了")
    print("\n上位5モデルが選択されました")
    
    # 最良モデルを選択
    best_model = best_models[0]
    
    print(f"\n🏆 最良モデル: {best_model.__class__.__name__}")
    
except Exception as e:
    print(f"\n❌ エラー: {e}")
    print("\n💡 PyCaretのセットアップが完了していない可能性があります")

🏆 モデル比較中...

※ 複数のアルゴリズムを試すため、数分かかります


## 🔧 9. モデルのチューニング

### 業務的な意義
モデルのパラメータを自動調整し、さらに精度を向上させる

In [ ]:
print("="*60)
print("🔧 モデルチューニング中...")
print("="*60)
print("\n※ この処理には数分かかる場合があります")

try:
    # モデルのチューニング
    tuned_model = tune_model(best_model, verbose=False)
    
    print("\n✅ チューニング完了")
    print("\n最適化されたモデルが構築されました")
    
except Exception as e:
    print(f"\n❌ エラー: {e}")
    print("\n💡 チューニングをスキップし、元のモデルを使用します")
    tuned_model = best_model

## 📈 10. モデル評価

### 業務的な意義
予測精度を確認し、実務で使えるレベルか判断する

In [ ]:
print("="*60)
print("📈 モデル評価")
print("="*60)

try:
    # テストデータで予測
    test_features = test_data[feature_cols].copy()
    test_features = test_features.dropna()
    
    predictions = predict_model(tuned_model, data=test_features)
    
    # 実際の値と予測値を比較
    actual = test_data.loc[predictions.index, target]
    predicted = predictions['prediction_label']
    
    # 評価指標の計算
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    
    mae = mean_absolute_error(actual, predicted)
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    r2 = r2_score(actual, predicted)
    
    print("\n【予測精度】")
    print(f"  MAE (平均絶対誤差): {mae:.2f}個")
    print(f"  RMSE (二乗平均平方根誤差): {rmse:.2f}個")
    print(f"  R² (決定係数): {r2:.3f}")
    
    print("\n" + "-"*60)
    print("【指標の読み方】")
    print(f"  MAE: 平均して{mae:.0f}個の誤差がある")
    print(f"  R²: {r2*100:.1f}%の精度で予測できている（1.0が完璧）")
    
    # 実測値 vs 予測値のグラフ
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.scatter(actual, predicted, alpha=0.6, color='steelblue')
    ax.plot([actual.min(), actual.max()], [actual.min(), actual.max()], 'r--', linewidth=2, label='完璧な予測')
    ax.set_title(f'実測値 vs 予測値 (R²={r2:.3f}, fontproperties=JP_FP)', fontsize=14)
    ax.set_xlabel('実測値（個）', fontsize=11, fontproperties=JP_FP)
    ax.set_ylabel('予測値（個）', fontsize=11, fontproperties=JP_FP)
    ax.legend(fontsize=11, prop=JP_FP)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"\n❌ エラー: {e}")

## 🎯 11. 特徴量重要度の分析

### 業務的な意義
どの要因が売上予測に最も影響しているか理解する

In [ ]:
print("="*60)
print("🎯 特徴量重要度分析")
print("="*60)

try:
    # 特徴量重要度のプロット
    plot_model(tuned_model, plot='feature', save=False)
    
    print("\n✅ 特徴量重要度を可視化しました")
    print("\n💡 上位の特徴量が予測に最も影響しています")
    
except Exception as e:
    print(f"\n⚠️ 特徴量重要度のプロットができませんでした: {e}")
    print("\n一部のモデルでは特徴量重要度が計算できない場合があります")

## 💾 12. モデルの保存

### 業務的な意義
構築したモデルを保存し、再利用できるようにする

In [ ]:
# 出力ディレクトリ
output_dir = Path('output/stage5_pycaret_models')
output_dir.mkdir(parents=True, exist_ok=True)

try:
    # モデルの保存
    model_path = output_dir / f'model_{demo_product.replace("/", "_")[:50]}'
    save_model(tuned_model, str(model_path))
    
    print("="*60)
    print("💾 モデル保存")
    print("="*60)
    print(f"\n✅ モデルを保存しました")
    print(f"   保存先: {model_path}.pkl")
    
    print("\n💡 次回はこのモデルを読み込んで予測できます")
    print("   読み込み: load_model('{}')".format(model_path))
    
except Exception as e:
    print(f"\n❌ モデル保存エラー: {e}")

## 🚀 13. 未来の予測（デモ）

### 業務的な意義
構築したモデルで明日・明後日の需要を予測し、発注量を決める

In [ ]:
print("="*60)
print("🚀 未来の需要予測（デモ）")
print("="*60)

print("\n※ 実際の予測システムは『動的売上予測システム - PyCaret 3.ipynb』で構築します")
print("\nここでは、テストデータの最初の5日分を予測例として表示します")

try:
    # テストデータの最初の5日分を予測
    sample_test = test_data.head(5).copy()
    sample_features = sample_test[feature_cols].dropna()
    
    # 予測実行
    sample_predictions = predict_model(tuned_model, data=sample_features)
    
    # 結果の整形
    result_df = pd.DataFrame({
        '日付': sample_test.loc[sample_predictions.index, '日付'].dt.strftime('%Y-%m-%d'),
        '実測値': sample_test.loc[sample_predictions.index, target].values,
        '予測値': sample_predictions['prediction_label'].values,
        '誤差': (sample_predictions['prediction_label'].values - 
                sample_test.loc[sample_predictions.index, target].values)
    })
    
    result_df['誤差率(%)'] = (result_df['誤差'] / result_df['実測値'] * 100).round(1)
    
    print("\n【予測結果サンプル】")
    print(result_df.to_string(index=False))
    
    print("\n💡 実務での使い方")
    print("  1. 翌日の天気・カレンダー情報を取得")
    print("  2. モデルで翌日の需要を予測")
    print("  3. 予測値に基づいて発注量を決定")
    print("  4. 安全在庫を加えた最終発注量を算出")
    
except Exception as e:
    print(f"\n❌ 予測エラー: {e}")

## 💡 14. Stage 5 まとめ

### PyCaretで実現できたこと
1. **自動モデル選択**: 複数のアルゴリズムを自動比較
2. **ハイパーパラメータ調整**: 最適なパラメータを自動探索
3. **高精度な予測**: 機械学習で需要を予測
4. **特徴量重要度**: どの要因が重要か理解
5. **モデルの保存**: 再利用可能な予測システム

### 実務での活用フロー
```
【日次の発注業務】
1. 翌日の天気予報・カレンダー情報を確認
2. PyCaretモデルで各商品の需要を予測
3. 予測値 + 安全在庫 = 発注量
4. 発注システムに入力
5. 実績を記録し、モデルを定期的に再学習
```

### 次のステップ
- **実運用システム**: 「動的売上予測システム - PyCaret 3.ipynb」で本格的な予測システムを構築
- **全商品への展開**: Aランク商品全てに予測モデルを適用
- **リアルタイム予測**: 日次で自動的に予測を更新
- **統合ダッシュボード**: 予測結果を可視化し、意思決定を支援

---

## 📝 重要な注意事項
このノートブックは基本的なPyCaretの使い方を示すデモです。

**実際の需要予測システムは別途構築します:**
- 「動的売上予測システム - PyCaret 3.ipynb」で詳細な予測システムを構築予定
- 複数商品の一括予測
- 時系列予測の高度なテクニック
- リアルタイム予測の自動化

---